# Chrome AI Built-in Prompt API Demo

This notebook demonstrates using Chrome's built-in AI (Gemini Nano) with the Prompt API.

**Requirements:**
- Chrome browser version 128+ 
- Enable experimental flags:
  - `chrome://flags/#prompt-api-for-gemini-nano`
  - `chrome://flags/#optimization-guide-on-device-model`
- First use requires downloading the Gemini Nano model

In [ ]:
import micropip
await micropip.install("anywidget")

In [ ]:
import anywidget
import traitlets

class ChromeAIWidget(anywidget.AnyWidget):
    _esm = """
    function render({ model, el }) {
      // Get accessor functions for traitlets
      let getPrompt = () => model.get("prompt");
      let getResponse = () => model.get("response");
      let getStatus = () => model.get("status");
      
      // Create status message
      let statusDiv = document.createElement("div");
      statusDiv.className = "status-message";
      statusDiv.textContent = getStatus();
      model.on("change:status", () => {
        statusDiv.textContent = getStatus();
        // Set color based on status
        const status = getStatus();
        if (status.includes("Error") || status.includes("not available")) {
          statusDiv.style.color = "#dc2626";
        } else if (status.includes("Generating") || status.includes("Creating")) {
          statusDiv.style.color = "#2563eb";
        } else if (status.includes("Ready") || status.includes("received")) {
          statusDiv.style.color = "#16a34a";
        } else {
          statusDiv.style.color = "inherit";
        }
      });
      el.appendChild(statusDiv);
      
      // Create prompt textarea
      let promptTextArea = document.createElement("textarea");
      promptTextArea.className = "prompt-input";
      promptTextArea.placeholder = "Enter your prompt here...";
      promptTextArea.rows = 3;
      promptTextArea.value = getPrompt();
      promptTextArea.addEventListener("keydown", (event) => {
        if (event.key === "Enter" && !event.shiftKey) {
          event.preventDefault();
          model.set("prompt", promptTextArea.value);
          model.save_changes();
        }
      });
      el.appendChild(promptTextArea);
      
      // Create submit button
      let button = document.createElement("button");
      button.className = "submit-button";
      button.textContent = "Submit";
      button.addEventListener("click", () => {
        model.set("prompt", promptTextArea.value);
        model.save_changes();
      });
      el.appendChild(button);
      
      // Create response textarea
      let responseTextArea = document.createElement("div");
      responseTextArea.className = "response-box";
      responseTextArea.textContent = getResponse();
      model.on("change:response", () => {
        responseTextArea.textContent = getResponse();
      });
      el.appendChild(responseTextArea);
      
      // Initialize Chrome AI session (async but doesn't block render)
      let session = null;
      (async () => {
        try {
          model.set("status", "Initializing...");
          model.save_changes();
          
          if (!window.ai || !window.ai.languageModel) {
            model.set("status", "Chrome AI not available. Please enable chrome://flags/#prompt-api-for-gemini-nano");
            model.save_changes();
            button.disabled = true;
            return;
          }
          
          model.set("status", "Creating AI session...");
          model.save_changes();
          session = await window.ai.languageModel.create();
          model.set("status", "Ready! Enter a prompt and press Submit or Enter.");
          model.save_changes();
          
        } catch (error) {
          model.set("status", `Error: ${error.message}`);
          model.save_changes();
          button.disabled = true;
        }
      })();
      
      // Handle prompt changes - call Chrome AI
      model.on("change:prompt", async () => {
        const promptText = getPrompt().trim();
        if (!promptText || !session) return;
        
        try {
          button.disabled = true;
          promptTextArea.disabled = true;
          
          model.set("status", "Generating response...");
          model.set("response", "");
          model.save_changes();
          
          const result = await session.prompt(promptText);
          
          model.set("response", result);
          model.set("status", "Response received! Enter another prompt.");
          model.save_changes();
          
        } catch (error) {
          model.set("response", `Error: ${error.message}`);
          model.set("status", "Error occurred. Please try again.");
          model.save_changes();
        } finally {
          button.disabled = false;
          promptTextArea.disabled = false;
        }
      });
    }
    export default { render };
    """
    
    _css = """
    .chrome-ai-container {
      font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
      max-width: 800px;
      margin: 0 auto;
      padding: 20px;
    }
    
    .status-message {
      padding: 12px;
      margin-bottom: 16px;
      border-radius: 6px;
      background-color: #f3f4f6;
      font-size: 14px;
      font-weight: 500;
    }
    
    .prompt-input {
      width: 100%;
      padding: 12px;
      margin-bottom: 12px;
      border: 2px solid #d1d5db;
      border-radius: 6px;
      font-size: 14px;
      font-family: inherit;
      resize: vertical;
      box-sizing: border-box;
    }
    
    .prompt-input:focus {
      outline: none;
      border-color: #3b82f6;
    }
    
    .submit-button {
      padding: 10px 20px;
      margin-bottom: 16px;
      background-color: #3b82f6;
      color: white;
      border: none;
      border-radius: 6px;
      font-size: 14px;
      font-weight: 500;
      cursor: pointer;
      transition: background-color 0.2s;
    }
    
    .submit-button:hover:not(:disabled) {
      background-color: #2563eb;
    }
    
    .submit-button:disabled {
      background-color: #9ca3af;
      cursor: not-allowed;
    }
    
    .response-box {
      padding: 16px;
      min-height: 100px;
      border: 2px solid #d1d5db;
      border-radius: 6px;
      background-color: #ffffff;
      white-space: pre-wrap;
      word-wrap: break-word;
      font-size: 14px;
      line-height: 1.6;
    }
    """
    
    prompt = traitlets.Unicode("").tag(sync=True)
    response = traitlets.Unicode("Response will appear here...").tag(sync=True)
    status = traitlets.Unicode("Initializing...").tag(sync=True)

## Create and Display the Widget

Create an instance of the Chrome AI widget and display it:

In [ ]:
ai_widget = ChromeAIWidget()
ai_widget

## Access Widget State from Python

The widget uses traitlets to synchronize state between JavaScript and Python. You can access all widget properties:

In [ ]:
print("Last prompt:", ai_widget.prompt)
print("Last response:", ai_widget.response)
print("Status:", ai_widget.status)